In [ ]:
from google.colab import drive
import scipy.io
import scipy.sparse as sp
import numpy as np
import os
import pandas as pd
import pprint

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the file path template and years range
file_path_template = '/content/drive/MyDrive/Yannis/EXIOBASE_3rx_aggLandUseExtensions_{year}_pxp.mat'
years = range(2004, 2016)

# Step 3: Load the .mat files
data_by_year = {}

for year in years:
    file_path = file_path_template.format(year=year)
    if os.path.exists(file_path):
        # Load the .mat file for the current year
        data = scipy.io.loadmat(file_path)

        # Extract the 'IO' structure and 'meta' structure
        io_data = data['IO']
        meta_data = data['meta'][0, 0]

        # Store the components of 'IO' into a dictionary
        data_by_year[year] = {
            'S': io_data['S'][0, 0] if 'S' in io_data.dtype.names else None,  # This matrix represents the transactions between different industries or sectors. Each entry Zij shows the flow from industry i to industry j.
            'A': io_data['A'][0, 0] if 'A' in io_data.dtype.names else None,  # The matrix of technical coefficients, also known as the direct requirements matrix. Each entry Aij shows the input from industry i required to produce one unit of output in industry j.
            'V': io_data['V'][0, 0] if 'V' in io_data.dtype.names else None,  # Represents value-added data or other economic vectors.
            'Y': io_data['Y'][0, 0] if 'Y' in io_data.dtype.names else None,  # The final demand matrix, showing the demand for goods and services from final consumers (households, government, etc.).
            'x': io_data['x'][0, 0] if 'x' in io_data.dtype.names else None,  # The total output vector, representing the total output produced by each industry.
            'TC': io_data['TC'][0, 0] if 'TC' in io_data.dtype.names else None,  # Could refer to trade coefficients or other related economic factors.
            'F': io_data['F'][0, 0] if 'F' in io_data.dtype.names else None,  # Factors of production or environmental stressors, including inputs like labor, capital, and natural resources.
            'F_hh': io_data['F_hh'][0, 0] if 'F_hh' in io_data.dtype.names else None,  # Stressors or impacts associated with household final demand.
            'pop': io_data['pop'][0, 0] if 'pop' in io_data.dtype.names else None,  # Population data, likely used to normalize or contextualize other data.
            'gdp': io_data['gdp'][0, 0] if 'gdp' in io_data.dtype.names else None,  # Gross Domestic Product data, representing the economic output.
            'VY': io_data['VY'][0, 0] if 'VY' in io_data.dtype.names else None  # This might contain additional vectors related to economic outputs and activities.
        }
    else:
        print(f"File for year {year} does not exist.")

import numpy as np

# Define the functions for each step
def calculate_carbon_intensity(emissions, output):
    return emissions / output

def calculate_external_exposure(exports, total_exports):
    return np.sum(exports) / total_exports

def calculate_fiscal_exposure(taxes, total_taxes):
    return np.sum(taxes) / total_taxes

def calculate_socioeconomic_exposure(employment, total_employment, wages, total_wages):
    employment_ratio = np.sum(employment) / total_employment
    wages_ratio = np.sum(wages) / total_wages
    return employment_ratio + wages_ratio

def calculate_indirect_dependence(input_coefficients, carbon_intensities):
    return np.sum(input_coefficients * carbon_intensities, axis=1)

def calculate_macroeconomic_exposure(external_exposure, fiscal_exposure, socioeconomic_exposure, alpha, beta, gamma):
    return alpha * external_exposure + beta * fiscal_exposure + gamma * socioeconomic_exposure

def get_row_labels(meta, key):
    if key in meta.dtype.names:
        row_labels = meta[key]
        return [label[0][0] if isinstance(label[0], np.ndarray) else label[0] for label in row_labels]
    else:
        print(f"{key} not found in metadata.")
        return None

# Assuming data_by_year is already populated from the previous steps
year = 2004
data = data_by_year[year]

# Extract relevant components
Z = data['S']  # Transactions matrix
A = data['A']  # Input coefficients matrix
Y = data['Y']  # Final demand matrix
x = data['x']  # Total output vector
F = data['F']  # Environmental stressors (e.g., emissions)
pop = data['pop']  # Population data
gdp = data['gdp']  # GDP data

# Function to get labels from meta_data
def get_labels(meta, key):
    if key in meta.dtype.names:
        labels = meta[key]
        return ['-'.join(map(str, label)) if isinstance(label, np.ndarray) else label for label in labels]
    else:
        print(f"{key} not found in metadata.")
        return None

# Get row and column labels
Z_labels = get_labels(meta_data, 'labsZ')
V_labels = get_labels(meta_data, 'labsV')
Y_labels = get_labels(meta_data, 'labsY')
F_labels = get_labels(meta_data, 'labsF')

# Convert matrices to DataFrames
Z_df = pd.DataFrame(Z.todense() if sp.issparse(Z) else Z)
A_df = pd.DataFrame(A.todense() if sp.issparse(A) else A)
Y_df = pd.DataFrame(Y.todense() if sp.issparse(Y) else Y)
x_df = pd.DataFrame(x)
F_df = pd.DataFrame(F.todense() if sp.issparse(F) else F)

# Get row labels for `F` matrix
F_row_labels = get_row_labels(meta_data, 'labsF')

# Print some row labels to verify
print("Sample F Row Labels: ", F_row_labels[:10])

# Check if 'F_hh' matrix is present and print its shape and some sample values
if data['F_hh'] is not None:
    print(f"Shape of F_hh matrix: {data['F_hh'].shape}")
    print("Sample F_hh Data: ", data['F_hh'][:5, :5])
else:
    print("'F_hh' matrix is not present.")

    # Define substrings for GHG emissions and their respective GWP values
ghg_substrings = {'CO2': 1, 'CH4': 83, 'N2O': 273}

# Identify the indices of the GHG emissions rows in F and apply GWP
ghg_emissions = np.zeros(F.shape[1])
for i, row_label in enumerate(F_row_labels):
    for ghg, gwp in ghg_substrings.items():
        if ghg in row_label:
            ghg_emissions += np.asarray(F[i, :].todense()).flatten() * gwp

# Print the shape and some sample values of the GHG emissions data
print(f"Shape of GHG emissions data from F: {ghg_emissions.shape}")
print("Sample GHG Emissions Data from F: ", ghg_emissions[:10])

# Check if F_hh matrix is present
if data['F_hh'] is not None:
    # Assuming F_hh has the same structure for row labels
    F_hh_row_labels = F_row_labels  # If F_hh has the same row labels as F

    # Identify the indices of the GHG emissions rows in F_hh and apply GWP
    ghg_emissions_hh = np.zeros(data['F_hh'].shape[1])
    for i, row_label in enumerate(F_hh_row_labels):
        for ghg, gwp in ghg_substrings.items():
            if ghg in row_label:
                ghg_emissions_hh += np.asarray(data['F_hh'][i, :].todense()).flatten() * gwp
else:
    ghg_emissions_hh = np.zeros((452, 1498))

# Print the shape and some sample values of the GHG emissions data from F_hh
print(f"Shape of GHG emissions data from F_hh: {ghg_emissions_hh.shape}")
print("Sample GHG Emissions Data from F_hh: ", ghg_emissions_hh[:10])

# Analyze emissions from F and F_hh separately

# Total GHG emissions from F
print(f"Shape of GHG emissions from F: {ghg_emissions.shape}")
print("Sample GHG Emissions from F: ", ghg_emissions[:10])

# Total GHG emissions from F_hh
print(f"Shape of GHG emissions from F_hh: {ghg_emissions_hh.shape}")
print("Sample GHG Emissions from F_hh: ", ghg_emissions_hh[:10])

Mounted at /content/drive
Sample F Row Labels:  ['Taxes less subsidies on products purchased: Total', 'Other net taxes on production', "Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled", "Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled", "Compensation of employees; wages, salaries, & employers' social contributions: High-skilled", 'Operating surplus: Consumption of fixed capital', 'Operating surplus: Rents on land', 'Operating surplus: Royalties on resources', 'Operating surplus: Remaining net operating surplus', 'Employment: Low-skilled male']
Shape of F_hh matrix: (452, 1498)
Sample F_hh Data:    (0, 0)	54.232491936563896
  (0, 1)	4.483002513379691
  (0, 2)	4.483002513379692
  (0, 3)	10.16726436889052
Shape of GHG emissions data from F: (42800,)
Sample GHG Emissions Data from F:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Shape of GHG emissions data from F_hh: (1498,)
Sample GHG Emissions Data from F_hh:  [0

In [ ]:
# Function to get labels from meta_data
def get_labels(meta, key):
    if key in meta.dtype.names:
        labels = meta[key]
        return ['-'.join(map(str, label)) if isinstance(label, np.ndarray) else label for label in labels]
    else:
        print(f"{key} not found in metadata.")
        return None

# Get row and column labels
Z_labels = get_labels(meta_data, 'labsZ')
V_labels = get_labels(meta_data, 'labsV')
Y_labels = get_labels(meta_data, 'labsY')
F_labels = get_labels(meta_data, 'labsF')

In [ ]:
# Print shapes and some content of the DataFrames
print("Z DataFrame Shape:", Z_df.shape)
print(Z_df.head())

print("A DataFrame Shape:", A_df.shape)
print(A_df.head())

print("Y DataFrame Shape:", Y_df.shape)
print(Y_df.head())

print("F DataFrame Shape:", F_df.shape)
print(F_df.head())

Z DataFrame Shape: (452, 42800)
      0         1         2         3         4         5         6      \
0  0.000046  0.001212  0.000214  0.001099  0.000169  0.000572  0.000239   
1  0.000055  0.002035  0.002462  0.002164  0.000602  0.001346  0.000643   
2  0.000719  0.000740  0.000718  0.001512  0.001931  0.006456  0.001209   
3  0.508853  0.545085  0.479199  0.586121  0.404070  0.375347  0.370782   
4  0.001012  0.001030  0.000991  0.002007  0.003279  0.007495  0.001977   

      7         8         9      ...     42790     42791     42792     42793  \
0  0.000674  0.001636  0.000320  ...  0.007635  0.006315  0.007823  0.006771   
1  0.003861  0.003868  0.000109  ...  0.012206  0.010212  0.012688  0.011139   
2  0.001117  0.000629  0.001113  ...  0.026090  0.021827  0.027119  0.023808   
3  0.500616  0.445684  0.581000  ...  0.141373  0.118273  0.146948  0.129008   
4  0.001654  0.000887  0.001694  ...  0.051543  0.043121  0.053576  0.047035   

      42794     42795     42796     

In [ ]:
!pip install pymrio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.2 MB/s eta 0:00:00
  Created wheel for pymrio: filename=pymrio-0.5.4-py3-none-any.whl size=140076 sha256=8a9f69217950893bd27b1d4a61d59b30c15ccb3ea726382bb9a9538828a20b16
  Stored in directory: /root/.cache/pip/wheels/f7/a2/98/de741b07dff4ebf8f79d17cd943f9d64f77c4fb2f1e08b7816
Successfully built pymrio
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.5
    Uninstalling openpyxl-3.1.5:
      Successfully uninstalled openpyxl-3.1.5


In [ ]:
import pymrio as py


FileNotFoundError: File parameter file /content/drive/MyDrive/Yannis/EXIOBASE_3rx_aggLandUseExtensions_2005_pxp.mat/file_parameters.json not found